In [1]:
!pip install beautifulsoup4
!python.exe -m pip install --upgrade pip

In [2]:
import requests,bs4,os,json,time

**fill in the member whose blog you want to download here**

In [3]:
desired_member_list = [
    ["current_new_森田 ひかる.json",'https://sakurazaka46.com/s/s46/diary/blog/list?ima=0000&ct=50'],
    ['current_new_大園 玲.json',"https://sakurazaka46.com/s/s46/diary/blog/list?ima=2951&ct=54"],
    ['current_new_三期生リレー.json','https://sakurazaka46.com/s/s46/diary/blog/list?ima=0000&ct=2000']
    
     
 ]

## download html

In [4]:
url = 'https://sakurazaka46.com/s/s46/diary/blog/list?ima=4714&page=0&ct=50&cd=blog'
html = requests.get(url)
time.sleep(1)
objSoup = bs4.BeautifulSoup(html.text,'lxml')
# html = requests.get(url)
# print('網頁下載中')
# html.raise_for_status()
# print('網頁下載成功')

## create Sakurazaka46 directory

In [5]:
deskDir = 'Sakurazaka46'
if os.path.exists(deskDir) == False:
    os.mkdir(deskDir)

## find blog links on current page

In [6]:
def get_next_page(current_objSoup):
    global current_page,download_complete
    
    box = current_objSoup.find('div', class_ = "com-pager wid1200")
    nextpage = box.find_all('li')
    for i in range(len(nextpage)):
        if nextpage[i].text == str(current_page+1):
            current_page += 1
            targetpage = nextpage[i]
            link = targetpage.find('a')['href']
            return url[:24] + link
    #it is end
    download_complete = True
    return ''

In [7]:
def download_pictures(blog_url,blog_index):
    global current_new,download_complete
    #url = https://www.nogizaka46.com/s/n46/diary/detail/100577?ima=2936&cd=MEMBER
    html = requests.get(blog_url)
    time.sleep(1)
    objSoup = bs4.BeautifulSoup(html.text,'lxml')
    
    box = objSoup.find('div',class_ = "box-article")
    imgTag = box.find_all('img')
    
    author_member = objSoup.find('p', class_ = "name").text
    
    blog_title = objSoup.find('h1', class_ = "title").text
    
    dateTime_raw = objSoup.find_all('p', class_ = "date wf-a")
    dateTime = dateTime_raw[1].text.split('/')
    date = dateTime[0] + dateTime[1] + dateTime[2][:2]
    
    
    data = [dateTime,author_member,blog_title]
    if data == current_new: #we can stop here
        download_complete = True
    else:
        #blog_title may not a good directory name
        blog_title = blog_title[:150]
        special_char = r'/\:*?"<>|'
        for i in special_char:
            blog_title = blog_title.replace(i,'')

        dir_path_member = './Sakurazaka46/' + author_member
        dir_path = './Sakurazaka46/' + author_member + '/' + date + ' ' + blog_title
        
        #the space at the end of blog_title should be remove
        dir_path = dir_path.rstrip(' .')
        
        if os.path.exists(dir_path_member) == False:
            os.mkdir(dir_path_member)
        if os.path.exists(dir_path) == False:
            os.mkdir(dir_path)
            if len(imgTag) > 0 :
                for i in range(len(imgTag)):
                    imgUrl = imgTag[i].get('src')

                    try:
                        is_effective_url = (not imgUrl is None) and ((imgUrl[-4:] in ['.jpg','.tif','.png']) or (imgUrl[-5:] in ['.jpeg']))
                        if is_effective_url: #we may catch something we don't want(no 'src') 
                            finUrl = url[:24] + imgUrl
                            
                            picture = requests.get(finUrl)
                            time.sleep(0.5)
                            picture.raise_for_status()
            #                 print('%s 圖片下載成功' %finUrl)

                            pictFile = open(os.path.join(dir_path,str(i).zfill(2) + (imgUrl[-4:] if imgUrl[-4:] in ['.jpg','.tif','.png'] else imgUrl[-5:])),'wb')
                            for diskStorage in picture.iter_content(1024*1024):
                                pictFile.write(diskStorage)
                            pictFile.close()
                    except requests.exceptions.HTTPError:
                        print(current_page,blog_index+1,'找不到網頁! 照片可能被鎖了',author_member,blog_title)
                        
                    except:
                        print(current_page,blog_index+1,'新的未知錯誤出現! 可能是異常個案',author_member,blog_title)
        else:
            print('看到重複的部落格了')

In [8]:
def update_current_new(current_new_url):
    html = requests.get(current_new_url)
    time.sleep(1)
    objSoup = bs4.BeautifulSoup(html.text,'lxml')
    
    blog_tags = objSoup.find_all('div', class_ = "c-blog-article__text")
    
    author_member = objSoup.find('p', class_ = "name").text
    
    blog_title = objSoup.find('h1', class_ = "title").text
    
    dateTime_raw = objSoup.find_all('p', class_ = "date wf-a")
    dateTime = dateTime_raw[1].text.split('/')
    date = dateTime[0] + dateTime[1] + dateTime[2][:2]
    
    current_new_data = [dateTime,author_member,blog_title]
    
    fn = 'current_new_' + author_member +'.json'
    with open(fn,'w',encoding='utf-8') as obj:
        json.dump(current_new_data,obj,ensure_ascii=False)

In [9]:
def download_blogs(current_objSoup):
    global current_new,current_page,download_complete
    boxes_raw = current_objSoup.find_all('li',class_ = 'box')
    boxes = []
    for i in boxes_raw:
        if i.find('div',class_ =  'wrap-bg') is not None:
            boxes.append(i)
    
    for i in range(len(boxes)):
        link = boxes[i].find('a')['href']
        blog_url = url[:24] + link
        
        if not download_complete:
            download_pictures(blog_url,i)
        
        if current_page == 1 and i == 0: #always update first
            update_current_new(blog_url)

**define main loop function**

In [10]:
def run_main_loop_for_desired_member(current_new_title,member_url):
    global current_new,current_page,download_complete
    fn = current_new_title

    try:
        with open(fn,'r',encoding='utf-8') as fnobj:
            current_new = json.load(fnobj)

    except Exception: #the 1st run of this code
        current_new = ''
        
    current_page = 1
    current_url = member_url

    download_complete = False
    while not download_complete:
        print(current_page)
        html = requests.get(current_url)
        current_objSoup = bs4.BeautifulSoup(html.text,'lxml')
        try:
            download_blogs(current_objSoup)
        except FileExistsError as e:
            print('檔案已存在! 載到重複的部落格')
            break
        except FileNotFoundError as e:
            print('檔案找不到! 可能部落格名稱太長或路徑有錯')
            break
        except NotADirectoryError as e:
            print('目錄名稱無效! 可能資料夾命名不合法')
            break
        except Exception as e:
            print( e,'可能為異常個案')
        current_url = get_next_page(current_objSoup)
        
    
    

****Run****

In [11]:
for member in desired_member_list:
    print(member[0])
    run_main_loop_for_desired_member(member[0],member[1])

current_new_森田 ひかる.json
1
current_new_大園 玲.json
1
current_new_三期生リレー.json
1


## get current new

In [12]:
#fn = 'current_new_森田 ひかる.json'
#
#try:
#    with open(fn,'r',encoding='utf-8') as fnobj:
#        current_new = json.load(fnobj)
#
#except Exception: #the 1st run of this code
#    current_new = ''

## main loop 森田 ひかる part

In [13]:
#current_page = 1
#current_url = 'https://sakurazaka46.com/s/s46/diary/blog/list?ima=4714&page=0&ct=50&cd=blog'
#download_complete = False
#while not download_complete:
#    print(current_page)
#    html = requests.get(current_url)
#    current_objSoup = bs4.BeautifulSoup(html.text,'lxml')
#    try:
#        download_blogs(current_objSoup)
#    except FileExistsError as e:
#        print('檔案已存在! 載到重複的部落格')
#        break
#    except FileNotFoundError as e:
#        print('檔案找不到! 可能部落格名稱太長或路徑有錯')
#        break
#    except NotADirectoryError as e:
#        print('目錄名稱無效! 可能資料夾命名不合法')
#        break
#    except Exception as e:
#        print( e,'可能為異常個案')
#    current_url = get_next_page(current_objSoup)

## get current new

In [14]:
#fn = 'current_new_大園 玲.json'
#
#try:
#    with open(fn,'r',encoding='utf-8') as fnobj:
#        current_new = json.load(fnobj)
#
#except Exception: #the 1st run of this code
#    current_new = ''

## main loop 大園 玲 part

In [15]:
#current_page = 1
#current_url = 'https://sakurazaka46.com/s/s46/diary/blog/list?ima=2951&ct=54'
#
#download_complete = False
#while not download_complete:
#    print(current_page)
#    html = requests.get(current_url)
#    current_objSoup = bs4.BeautifulSoup(html.text,'lxml')
#    try:
#        download_blogs(current_objSoup)
#    except FileExistsError as e:
#        print('檔案已存在! 載到重複的部落格')
#        break
#    except FileNotFoundError as e:
#        print('檔案找不到! 可能部落格名稱太長或路徑有錯')
#        break
#    except NotADirectoryError as e:
#        print('目錄名稱無效! 可能資料夾命名不合法')
#        break
#    except Exception as e:
#        print( e,'可能為異常個案')
#    current_url = get_next_page(current_objSoup)

**三期生relay**

In [16]:
fn = 'current_new_三期生リレー.json'

try:
    with open(fn,'r',encoding='utf-8') as fnobj:
        current_new = json.load(fnobj)

except Exception: #the 1st run of this code
    current_new = ''

In [17]:
current_page = 1
current_url = 'https://sakurazaka46.com/s/s46/diary/blog/list?ima=0000&ct=2000'

download_complete = False
while not download_complete:
    print(current_page)
    html = requests.get(current_url)
    current_objSoup = bs4.BeautifulSoup(html.text,'lxml')
    try:
        download_blogs(current_objSoup)
    except FileExistsError as e:
        print('檔案已存在! 載到重複的部落格')
        break
    except FileNotFoundError as e:
        print('檔案找不到! 可能部落格名稱太長或路徑有錯')
        break
    except NotADirectoryError as e:
        print('目錄名稱無效! 可能資料夾命名不合法')
        break
    except Exception as e:
        print( e,'可能為異常個案')
    current_url = get_next_page(current_objSoup)

1
